In [1]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import random
import time
import datetime

In [2]:
# Esta clase engloba a las cajas y a los estantes en una sola representacion numerica 
class Cell(Agent):
   
  def __init__(self, id, model, cell_type, num_boxes):
     
    super().__init__(id, model)
    self.id = id

    # Tipo de la celda 0: Pared 1: Piso 2: Pila
    self.cell_type = cell_type
    self.boxes_here = num_boxes

In [3]:
 
class Robot(Agent):
  # Lista que contiene todos los movimientos posibles del robot al intentar rodear un obstaculo
  moves = [(-1,-1), (0,-1), (1,-1), (1,0), (1,1), (0,1), (-1,1), (-1,0)]
  move_counter = 0

  def __init__(self, id, model, spawn, end):

    super().__init__(id, model)
    self.id = id


    self.pos = spawn
    self.destination = end
    self.end_pos = end

    self.assigned_box = None
    self.assigned_stack = None
    self.carrying_box = False
    self.active = True


  def step(self):
    # Si ya acabó no hace nada 
    if not(self.active): return

    # Revisa si se puede mover en diagonal o en linea recta segun la diferencia 
    # en x y con la posicion de su objetivo 
    dx = self.destination[0] - self.pos[0]
    dy = self.destination[1] - self.pos[1]
    move = (0 if dx == 0 else dx // abs(dx), 0 if dy == 0 else dy // abs(dy))
    next_pos = (self.pos[0] + move[0], self.pos[1] + move[1])

    # En casoo de que acabamos de llegar a una caja que tenemos que recojer
    if next_pos == self.assigned_box and not(self.carrying_box):
      
      #Decimos que ya estamos cargando la caja y ahora vamos hacia el estante
      self.carrying_box = True
      self.assigned_box = None
      self.destination = self.assigned_stack
      self.model.grid.get_cell_list_contents(next_pos)[0].boxes_here = 0
    # En caso de que acabamos de llegar al estante donde se va a dejar la caja
    elif next_pos == self.assigned_stack and self.carrying_box:
      # Decimos que ya no estamos cargando la caja, estamos disponibles
      #y se deja la caja en el estante
      self.carrying_box = False
      self.assigned_stack = None
      self.destination = self.end_pos
      #aqui se apila la caja en si 
      self.model.grid.get_cell_list_contents(next_pos)[0].boxes_here += 1
      #aqui dice que ya esta libre
      self.model.robots_free.add(self)
    elif move != (0,0):
      # En caso de que en la siguiente posicion hay un obstaculo
      agents_there = self.model.grid.get_cell_list_contents(next_pos)
      
      # revisa si es que necesitamos rodear, y llama a la funcion para rodear
      if (len(agents_there) > 1 or agents_there[0].cell_type in [0,2] or agents_there[0].boxes_here > 0):
        next_pos = self.go_around(next_pos, move)
      
      # avanzamos a la siguiente posicion
      self.model.grid.move_agent(self, next_pos)
      self.pos = next_pos
      Robot.move_counter += 1

    # Revisar si ya esta estacionado 
    if self.pos == self.end_pos and self.destination == self.end_pos:
      self.model.active_robots -= 1
      self.active = False


  
  # Aqui el robot asigna una caja que ganó en la subasta
  def assign_box(self, box_pos, stack_pos):
    self.destination = box_pos
    self.assigned_box = box_pos
    self.assigned_stack = stack_pos

  # Esta es la funcion para rodear 
  def go_around(self, next_pos, move):
    # Esto es equivalente a usar un INT_MAX o INF como valor inicial de costo 
    move_cost = self.model.n + self.model.m + 1
    move_idx = self.moves.index(move)

    # Solo se intenta 4 veces, porque en realidad tras 4 iteraciones 
    # el robot va a terminar de explorar todas las celdas a su alrededor, y si no encuentra
    # hacia donde moverse en esas iteraciones, es porque no hay hacia donde moverse y esta atrapado.
    # entonces se tiene que esperar a que se libere el camino (algun otro robot se movera o movera las cajas)
    for rotations in range(1, 5):
      #escoge un movmimeinto del set de movimientos
      right_move = self.moves[(move_idx + rotations) % len(self.moves)]
      left_move = self.moves[(move_idx - rotations + len(self.moves)) % len(self.moves)]

      # calcula la posicion nueva
      right_pos = (self.pos[0] + right_move[0], self.pos[1] + right_move[1])
      left_pos = (self.pos[0] + left_move[0], self.pos[1] + left_move[1])

      # calcula cuanto le costaria moverse a dicha posicion
      right_cost = self.go_around_cost(right_pos, move_cost)
      left_cost = self.go_around_cost(left_pos, move_cost)

      # busca cual es la opcion mas economica, siempre y cuando al menos una sea valida
      if right_cost != move_cost or left_cost != move_cost:
        min_cost = min(right_cost, left_cost)
        return right_pos if min_cost == right_cost else left_pos
    
    #esto ya solo es en caso de que no se pudo mover, se espera en su misma pos
    return self.pos

  def go_around_cost(self, new_pos, default_cost):
     #ayuda a saber el costo de rodear
    agents_there = self.model.grid.get_cell_list_contents(new_pos)
    if (len(agents_there) > 1 or agents_there[0].cell_type in [0,2] or agents_there[0].boxes_here > 0):
      return default_cost
    return self.model.distance(new_pos, self.destination)

In [4]:
def get_grid(model):

  grid = np.zeros((model.grid.width, model.grid.height))
  for (cell_content, x, y) in model.grid.coord_iter():
    if len(cell_content) > 1:
      grid[x][y] = 7
    elif cell_content[0].cell_type == 0:
      grid[x][y] = 8
    elif cell_content[0].boxes_here > 0:
      grid[x][y] = cell_content[0].boxes_here + 1
    else:
      grid[x][y] = cell_content[0].cell_type - 1
  return np.transpose(grid) #aqui arreglo x,y para que queden normales ._. estaban al reves

In [5]:
class StorageModel(Model):
  # Constructor
  def __init__(self, m, n, num_robots, num_boxes):
    self.m = m
    self.n = n
    self.num_robots = num_robots
    self.active_robots = num_robots
    self.num_boxes = num_boxes
    
    self.grid = MultiGrid(m + 2, n + 2, False)
    self.schedule = SimultaneousActivation(self)
    self.datacollector = DataCollector(model_reporters = {"Grid": get_grid})
    num_stacks = int(np.ceil(self.num_boxes / 5))
    self.stacks_pos = [(i % n + 1, i // n + 1) for i in range(num_stacks)]
    self.boxes_left = set()
    self.robots_free = set()
    self.stack_to_assign = [0, 0]

    #Aqui se construye el escenario
    for (content, x, y) in self.grid.coord_iter():
      if x in [0, m + 1] or y in [0, n + 1]:
        new_cell = Cell((x,y), self, 0, 0)
      elif (x,y) in self.stacks_pos:
        new_cell = Cell((x,y), self, 2, 0)
      else:
        new_cell = Cell((x,y), self, 1, 0)
      self.grid.place_agent(new_cell, (x,y))
    
    
    free_cells = [(x, y) for (content, x, y) in self.grid.coord_iter() if (x,y)
      not in self.stacks_pos and x not in [0, m + 1] and y not in [0, n + 1]]
    selected_spawns = random.sample(free_cells, self.num_boxes + self.num_robots)

    #se definen los ptos de estacionar para los robots
    robot_ends = [(n - (i % n), m - (i // n)) for i in range(self.num_robots)]

    #Aqui spawnea a los robots
    for i in range(self.num_robots):
      new_robot = Robot(i, self, selected_spawns[i], robot_ends[i])
      self.grid.place_agent(new_robot, (selected_spawns[i]))
      self.schedule.add(new_robot)
      self.robots_free.add(new_robot)
    
    # spawnea las cajas
    for i in range(self.num_robots, len(selected_spawns)):
      self.grid.get_cell_list_contents(selected_spawns[i])[0].boxes_here = 1
      self.boxes_left.add(selected_spawns[i])

    
    self.datacollector.collect(self)

 
  def step(self):
    #primero se hace la subasta y ya luego todo lo demás
    self.auction_boxes()
    self.schedule.step()
    self.datacollector.collect(self)

  def auction_boxes(self):
  
    for box_pos in self.boxes_left.copy():
      if len(self.robots_free) == 0: 
        return

      chosen_robot = None
      task_cost = self.m + self.n + 1
      for robot in self.robots_free:
        robot_cost = (self.distance(robot.pos, box_pos) + 
          self.distance(box_pos, self.stacks_pos[self.stack_to_assign[0]]))
        # El robot mas economico gana la subasta (el que haga menos movs)
        if task_cost > robot_cost:
          task_cost = robot_cost
          chosen_robot = robot
      
      # asigna a la caja, revisa si el estante ya se llenó para pasar al que sigue
      chosen_robot.assign_box(box_pos, self.stacks_pos[self.stack_to_assign[0]])
      self.boxes_left.remove(box_pos)
      self.robots_free.remove(chosen_robot)
      self.stack_to_assign[1] += 1
      if self.stack_to_assign[1] == 5:
        self.stack_to_assign = [self.stack_to_assign[0] + 1, 0]

  
  def distance(self, pos1, pos2):
    dx = abs(pos1[0] - pos2[0])
    dy = abs(pos1[1] - pos2[1])
    return max(dx, dy)

In [6]:

M = 16
N = 16
NUM_ROBOTS = 5
NUM_BOXES = 40
MAX_DURATION = 0.1


start_time = time.time()
 
model = StorageModel(M, N, NUM_ROBOTS, NUM_BOXES)
while time.time() - start_time < MAX_DURATION and model.active_robots > 0:
  model.step()


duration = time.time() - start_time




In [7]:

all_grids = model.datacollector.get_model_vars_dataframe()

oTiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas. 


o Número de movimientos realizados por todos los robots. 


In [8]:


print ("Resultados de la ejecución:")
if (duration >= MAX_DURATION):
  print(f"Tiempo de ejecución: {MAX_DURATION}s (Se alcanzó la duración máxima permitida)")
else:
  print(f"Tiempo de ejecución: {round(duration, 3)}s (Se acomodaron todas las cajas antes de que se acabara el tiempo)")

 
print("Movimientos totales de Robot: ", Robot.move_counter)


Resultados de la ejecución:
Tiempo de ejecución: 0.053s (Se acomodaron todas las cajas antes de que se acabara el tiempo)
Movimientos totales de Robot:  881


In [13]:
%%capture

old_cmap = matplotlib.cm.get_cmap('viridis', 9)
old_colors = old_cmap(np.linspace(0, 1, 9))

# Colores en formato RGBA (como los de unity )
old_colors[0] = np.array([256/256, 256/256, 256/256, 1]) # Blanco  
old_colors[1] = np.array([200/256, 200/256, 200/256, 1]) # Gris claro 
old_colors[2] = np.array([178/256, 155/256, 129/256, 1]) # Café claro  
old_colors[3] = np.array([152/256, 128/256, 100/256, 1]) # Café claro++
old_colors[4] = np.array([126/256, 101/256, 73/256, 1]) # Café
old_colors[5] = np.array([99/256, 73/256, 44/256, 1]) # Café oscuro
old_colors[6] = np.array([73/256, 46/256, 16/256, 1]) # Café oscuro++
old_colors[7] = np.array([27/256, 52/256, 25/256, 1]) # Verde 
old_colors[8] = np.array([0, 0, 0, 1]) # Negro

new_cmap = matplotlib.colors.ListedColormap(old_colors)

 
plt.rcParams["animation.html"] = "jshtml"
 
matplotlib.rcParams['animation.embed_limit'] = 2**128

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grids.iloc[0][0], cmap=new_cmap, extent=[0, M + 2, N + 2, 0])

def animate(i):
    patch.set_data(all_grids.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames = len(all_grids))

In [14]:
anim

# Mi análisis


#### o Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como 
#### la cantidad de movimientos realizados. ¿Cómo sería? Descríbela. 

Sí. Creo que esta actividad fue un reto muy interesante, muy enfocado a la optimización sobre todo. 

Al analizar las posibilidades de implementación noté que la forma de reducir 
la cantidad de movimientos era precisamente basar la asiganción de cajas en la 
distancia (en movimientos, no euclidiana) a la que se encontraba cada robot. 
Esto permite que siempre cada caja tenga al robot que hará los menos movimientos posibles. 

Además esto mismo permite que los robots acaben en un tiempo menor, ya que 
al reducir la cantidad de movs. le tomará a los robots menos frames acabar su tarea.

Muy probablemente existan formas de optimizar aún mas implementando algoritmos
de busqueda mas avanzados, algoritmos como el shortest spanning tree pueden
ser de mucha utilidad. 

Algo más a considerar es que en la vida real, los robots no se pueden quedar
parados a medio almacén, deben estacionarse en sus estaciones de carga, y esto
implica sumar unos cuantos movimientos adicionales al final. 

